In this tutorial, we build a simple two tower ranking model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TF-Ranking. We can use this model to rank and recommend movies for a given user according to their predicted user ratings.

## Setup

Install and import the TF-Ranking library:

In [1]:
# !pip install -q tensorflow-ranking
# !pip install -q --upgrade tensorflow-datasets

In [18]:
from typing import Dict, Tuple

import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_ranking as tfr

## Read the data

Prepare to train a model by creating a ratings dataset and movies dataset. Use `user_id` as the query input feature, `movie_title` as the document input feature, and `user_rating` as the label to train the ranking model.

In [2]:
%%capture --no-display
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

2023-06-10 04:58:31.165224: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
# movies

Build vocabularies to convert all user ids and all movie titles into integer indices for embedding layers:

In [4]:
movies = movies.map(lambda x: x["movie_title"])
users = ratings.map(lambda x: x["user_id"])

user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(
    mask_token=None)
user_ids_vocabulary.adapt(users.batch(1000))

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(
    mask_token=None)
movie_titles_vocabulary.adapt(movies.batch(1000))

2023-06-10 04:58:32.617479: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-06-10 04:58:32.617812: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-06-10 04:58:34.650830: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{n

In [5]:
movie_titles_vocabulary

Group by `user_id` to form lists for ranking models:


In [6]:
key_func = lambda x: user_ids_vocabulary(x["user_id"])
reduce_func = lambda key, dataset: dataset.batch(100)
ds_train = ratings.group_by_window(
    key_func=key_func, reduce_func=reduce_func, window_size=100)

In [7]:
for x in ds_train.take(1):
  for key, value in x.items():
    print(f"Shape of {key}: {value.shape}")
    print(f"Example values of {key}: {value[:5].numpy()}")
    print()

2023-06-10 04:58:34.781244: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-06-10 04:58:34.781583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]


Shape of movie_title: (100,)
Example values of movie_title: [b'Man Who Would Be King, The (1975)' b'Silence of the Lambs, The (1991)'
 b'Next Karate Kid, The (1994)' b'2001: A Space Odyssey (1968)'
 b'Usual Suspects, The (1995)']

Shape of user_id: (100,)
Example values of user_id: [b'405' b'405' b'405' b'405' b'405']

Shape of user_rating: (100,)
Example values of user_rating: [1. 4. 1. 5. 5.]



Generate batched features and labels:

In [8]:
def _features_and_labels(
    x: Dict[str, tf.Tensor]) -> Tuple[Dict[str, tf.Tensor], tf.Tensor]:
  labels = x.pop("user_rating")
  return x, labels


ds_train = ds_train.map(_features_and_labels)

ds_train = ds_train.apply(
    tf.data.experimental.dense_to_ragged_batch(batch_size=32))

Instructions for updating:
Use `tf.data.Dataset.ragged_batch` instead.


The `user_id` and `movie_title` tensors generated in `ds_train` are of shape `[32, None]`, where the second dimension is 100 in most cases except for the batches when less than 100 items grouped in lists. A model working on ragged tensors is thus used.

In [9]:
for x, label in ds_train.take(1):
  for key, value in x.items():
    print(f"Shape of {key}: {value.shape}")
    print(f"Example values of {key}: {value[:3, :3].numpy()}")
    print()
  print(f"Shape of label: {label.shape}")
  print(f"Example values of label: {label[:3, :3].numpy()}")

2023-06-10 04:58:36.435103: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-06-10 04:58:36.435463: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]


Shape of movie_title: (32, None)
Example values of movie_title: [[b'Man Who Would Be King, The (1975)'
  b'Silence of the Lambs, The (1991)' b'Next Karate Kid, The (1994)']
 [b'Flower of My Secret, The (Flor de mi secreto, La) (1995)'
  b'Little Princess, The (1939)' b'Time to Kill, A (1996)']
 [b'Kundun (1997)' b'Scream (1996)' b'Power 98 (1995)']]

Shape of user_id: (32, None)
Example values of user_id: [[b'405' b'405' b'405']
 [b'655' b'655' b'655']
 [b'13' b'13' b'13']]

Shape of label: (32, None)
Example values of label: [[1. 4. 1.]
 [3. 3. 3.]
 [5. 1. 1.]]


## Define a model

Define a ranking model by inheriting from `tf.keras.Model` and implementing the `call` method:

In [10]:
class MovieLensRankingModel(tf.keras.Model):

  def __init__(self, user_vocab, movie_vocab):
    super().__init__()

    # Set up user and movie vocabulary and embedding.
    self.user_vocab = user_vocab
    self.movie_vocab = movie_vocab
    self.user_embed = tf.keras.layers.Embedding(user_vocab.vocabulary_size(),
                                                64)
    self.movie_embed = tf.keras.layers.Embedding(movie_vocab.vocabulary_size(),
                                                 64)

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    # Define how the ranking scores are computed: 
    # Take the dot-product of the user embeddings with the movie embeddings.

    user_embeddings = self.user_embed(self.user_vocab(features["user_id"]))
    movie_embeddings = self.movie_embed(
        self.movie_vocab(features["movie_title"]))

    return tf.reduce_sum(user_embeddings * movie_embeddings, axis=2)

Create the model, and then compile it with ranking `tfr.keras.losses` and `tfr.keras.metrics`, which are the core of the TF-Ranking package. 

This example uses a ranking-specific **softmax loss**, which is a listwise loss introduced to promote all relevant items in the ranking list with better chances on top of the irrelevant ones. In contrast to the softmax loss in the multi-class classification problem, where only one class is positive and the rest are negative, the TF-Ranking library supports multiple relevant documents in a query list and non-binary relevance labels.

For ranking metrics, this example uses in specific **Normalized Discounted Cumulative Gain (NDCG)** and **Mean Reciprocal Rank (MRR)**, which calculate the user utility of a ranked query list with position discounts. For more details about ranking metrics, review evaluation measures [offline metrics](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Offline_metrics).

### Point-wise loss 

In [12]:


loss = tfr.keras.losses.MeanSquaredLoss(ragged=True)


model = MovieLensRankingModel(user_ids_vocabulary, movie_titles_vocabulary)
optimizer = tf.keras.optimizers.Adagrad(0.5)
eval_metrics = [
    tfr.keras.metrics.get(key="ndcg", name="metric/ndcg", ragged=True),
    tfr.keras.metrics.get(key="mrr", name="metric/mrr", ragged=True)
]
model.compile(optimizer=optimizer, loss=loss, metrics=eval_metrics)
model.fit(ds_train, epochs=3)

2023-06-10 04:58:44.017131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-06-10 04:58:44.017484: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_23' with dtype resource
	 [[{{node Placeholder/_23}}]]


Epoch 1/3


2023-06-10 04:58:44.353548: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'movie_lens_ranking_model/cond/zeros/Reshape/movie_lens_ranking_model/sub_1' with dtype int64
	 [[{{node movie_lens_ranking_model/cond/zeros/Reshape/movie_lens_ranking_model/sub_1}}]]
2023-06-10 04:58:44.357588: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'movie_lens_ranking_model/cond/cond/range/movie_lens_ranking_model/sub' with dtype int64
	 [[{{node movie_lens_ranking_model/cond/cond/range/movie_lens_ranking_model/sub}}]]
2023-06-10 04:58:44.362080: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor s

48/48 [==============================] - 7s 78ms/step - loss: 12.5849 - metric/ndcg: 0.8100 - metric/mrr: 1.0000
Epoch 2/3
48/48 [==============================] - 5s 73ms/step - loss: 12.5794 - metric/ndcg: 0.8160 - metric/mrr: 1.0000
Epoch 3/3
48/48 [==============================] - 5s 68ms/step - loss: 12.5703 - metric/ndcg: 0.8254 - metric/mrr: 1.0000


### Pair-wise loss 

In [13]:
# # Create the ranking model, trained with a ranking loss and evaluated with
# # ranking metrics.

# loss = tfr.keras.losses.PairwiseHingeLoss(ragged=True)


# model = MovieLensRankingModel(user_ids_vocabulary, movie_titles_vocabulary)
# optimizer = tf.keras.optimizers.Adagrad(0.5)
# eval_metrics = [
#     tfr.keras.metrics.get(key="ndcg", name="metric/ndcg", ragged=True),
#     tfr.keras.metrics.get(key="mrr", name="metric/mrr", ragged=True)
# ]
# model.compile(optimizer=optimizer, loss=loss, metrics=eval_metrics)
# model.fit(ds_train, epochs=3)

### List-wise loss 

In [14]:
# # Create the ranking model, trained with a ranking loss and evaluated with
# # ranking metrics.

# loss = tfr.keras.losses.get(
#     loss=tfr.keras.losses.RankingLossKey.SOFTMAX_LOSS, ragged=True)

# model = MovieLensRankingModel(user_ids_vocabulary, movie_titles_vocabulary)
# optimizer = tf.keras.optimizers.Adagrad(0.5)
# eval_metrics = [
#     tfr.keras.metrics.get(key="ndcg", name="metric/ndcg", ragged=True),
#     tfr.keras.metrics.get(key="mrr", name="metric/mrr", ragged=True)
# ]
# model.compile(optimizer=optimizer, loss=loss, metrics=eval_metrics)
# model.fit(ds_train, epochs=3)

## Train and evaluate the model

Train the model with `model.fit`.

Generate predictions and evaluate.

In [15]:
# Get movie title candidate list.
for movie_titles in movies.batch(2000):
  break

# Generate the input for user 42.
inputs = {
    "user_id":
        tf.expand_dims(tf.repeat("42", repeats=movie_titles.shape[0]), axis=0),
    "movie_title":
        tf.expand_dims(movie_titles, axis=0)
}

# Get movie recommendations for user 42.
scores = model(inputs)
titles = tfr.utils.sort_by_scores(scores,
                                  [tf.expand_dims(movie_titles, axis=0)])[0]
print(f"Top 5 recommendations for user 42: {titles[0, :5]}")

Top 5 recommendations for user 42: [b'First Kid (1996)' b'Larger Than Life (1996)'
 b'Naked in New York (1994)' b'Man from Down Under, The (1943)'
 b'Jack (1996)']


2023-06-10 04:59:08.749824: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-06-10 04:59:08.750381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]


# Ranking pipeline example 

Refer to [github page](https://github.com/tensorflow/ranking/blob/master/tensorflow_ranking/examples/tf_ranking_canned_dnn.py) for more details 

In [16]:
# # Copyright 2022 The TensorFlow Ranking Authors.
# #
# # Licensed under the Apache License, Version 2.0 (the "License");
# # you may not use this file except in compliance with the License.
# # You may obtain a copy of the License at
# #
# #     http://www.apache.org/licenses/LICENSE-2.0
# #
# # Unless required by applicable law or agreed to in writing, software
# # distributed under the License is distributed on an "AS IS" BASIS,
# # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# # See the License for the specific language governing permissions and
# # limitations under the License.

# r"""TF-Ranking example code for training a canned DNN estimator.

# The supported proto formats are listed at ../python/data.py.
# --------------------------------------------------------------------------------
# Sample command lines:

# MODEL_DIR=/tmp/output && \
# TRAIN=tensorflow_ranking/examples/data/train_numerical_elwc.tfrecord && \
# EVAL=tensorflow_ranking/examples/data/vali_numerical_elwc.tfrecord && \
# rm -rf $MODEL_DIR && \
# bazel build -c opt \
# tensorflow_ranking/examples/tf_ranking_canned_dnn_py_binary && \
# ./bazel-bin/tensorflow_ranking/examples/tf_ranking_canned_dnn_py_binary \
# --train_input_pattern=$TRAIN \
# --eval_input_pattern=$EVAL \
# --model_dir=$MODEL_DIR

# You can use TensorBoard to display the training results stored in $MODEL_DIR.

# Notes:
#   * Use --alsologtostderr if the output is not printed into screen.
# """

# from absl import flags

# import tensorflow as tf
# import tensorflow_ranking as tfr

# flags.DEFINE_string("train_input_pattern", None,
#                     "Input file path used for training.")
# flags.DEFINE_string("eval_input_pattern", None,
#                     "Input file path used for eval.")
# flags.DEFINE_string("model_dir", None, "Output directory for models.")
# flags.DEFINE_integer("batch_size", 32, "The batch size for train.")
# flags.DEFINE_integer("num_train_steps", 15000, "Number of steps for train.")
# flags.DEFINE_integer("num_eval_steps", 10, "Number of steps for evaluation.")
# flags.DEFINE_integer("checkpoint_secs", 30,
#                      "Saves a model checkpoint every checkpoint_secs seconds.")
# flags.DEFINE_integer("num_checkpoints", 100,
#                      "Saves at most num_checkpoints checkpoints in workspace.")
# flags.DEFINE_integer("num_features", 136, "Number of features per example.")
# flags.DEFINE_integer(
#     "list_size", 100,
#     "List size used for training. Use None for dynamic list size.")
# flags.DEFINE_float("learning_rate", 0.05, "Learning rate for optimizer.")
# flags.DEFINE_float("dropout", 0.8, "The dropout rate before output layer.")
# flags.DEFINE_list("hidden_layer_dims", ["64", "32", "16"],
#                   "Sizes for hidden layers.")
# flags.DEFINE_string("loss", "approx_ndcg_loss",
#                     "The RankingLossKey for the loss function.")
# flags.DEFINE_bool("convert_labels_to_binary", False,
#                   "If true, relevance labels are set to either 0 or 1.")
# flags.DEFINE_bool("listwise_inference", False,
#                   "If true, exports accept `data_format` while serving.")

# FLAGS = flags.FLAGS

# _LABEL_FEATURE = "utility"


# def context_feature_columns():
#   """Returns context feature columns."""
#   return {}


# def example_feature_columns():
#   """Returns the example feature columns."""
#   feature_names = [
#       "custom_features_{}".format(i + 1) for i in range(FLAGS.num_features)
#   ]
#   return {
#       name:
#       tf.feature_column.numeric_column(name, shape=(1,), default_value=0.0)
#       for name in feature_names
#   }


# def train_and_eval():
#   """Train and Evaluate."""
#   optimizer = tf.compat.v1.train.AdagradOptimizer(
#       learning_rate=FLAGS.learning_rate)

#   estimator = tfr.estimator.make_dnn_ranking_estimator(
#       example_feature_columns(),
#       FLAGS.hidden_layer_dims,
#       context_feature_columns=context_feature_columns(),
#       optimizer=optimizer,
#       learning_rate=FLAGS.learning_rate,
#       loss=FLAGS.loss,
#       loss_reduction=tf.compat.v1.losses.Reduction.SUM_OVER_BATCH_SIZE,
#       activation_fn=tf.nn.relu,
#       dropout=FLAGS.dropout,
#       use_batch_norm=True,
#       model_dir=FLAGS.model_dir)

#   hparams = {"train_input_pattern": FLAGS.train_input_pattern,
#              "eval_input_pattern": FLAGS.eval_input_pattern,
#              "learning_rate": FLAGS.learning_rate,
#              "train_batch_size": FLAGS.batch_size,
#              "eval_batch_size": FLAGS.batch_size,
#              "predict_batch_size": FLAGS.batch_size,
#              "num_train_steps": FLAGS.num_train_steps,
#              "num_eval_steps": FLAGS.num_eval_steps,
#              "checkpoint_secs": FLAGS.checkpoint_secs,
#              "num_checkpoints": FLAGS.num_checkpoints,
#              "loss": FLAGS.loss,
#              "list_size": FLAGS.list_size,
#              "convert_labels_to_binary": FLAGS.convert_labels_to_binary,
#              "listwise_inference": FLAGS.listwise_inference,
#              "model_dir": FLAGS.model_dir}

#   ranking_pipeline = tfr.ext.pipeline.RankingPipeline(
#       context_feature_columns(),
#       example_feature_columns(),
#       hparams,
#       estimator=estimator,
#       label_feature_name=_LABEL_FEATURE,
#       label_feature_type=tf.int64)

#   ranking_pipeline.train_and_eval()


# def main(_):
#   tf.compat.v1.set_random_seed(1234)
#   tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
#   train_and_eval()


# if __name__ == "__main__":
#   flags.mark_flag_as_required("train_input_pattern")
#   flags.mark_flag_as_required("eval_input_pattern")
#   flags.mark_flag_as_required("model_dir")

#   tf.compat.v1.app.run()

In [17]:
# # Copyright 2022 The TensorFlow Ranking Authors.
# #
# # Licensed under the Apache License, Version 2.0 (the "License");
# # you may not use this file except in compliance with the License.
# # You may obtain a copy of the License at
# #
# #     http://www.apache.org/licenses/LICENSE-2.0
# #
# # Unless required by applicable law or agreed to in writing, software
# # distributed under the License is distributed on an "AS IS" BASIS,
# # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# # See the License for the specific language governing permissions and
# # limitations under the License.

# """Tests for tf_ranking_canned_dnn.py."""

# import os

# from absl import flags
# from absl.testing import flagsaver
# from absl.testing import parameterized

# import tensorflow as tf

# from google.protobuf import text_format
# from tensorflow_ranking.examples import tf_ranking_canned_dnn
# from tensorflow_serving.apis import input_pb2

# FLAGS = flags.FLAGS

# ELWC = text_format.Parse(
#     """
#     context {
#     }
#     examples {
#       features {
#         feature {
#           key: "custom_features_1"
#           value { float_list { value: 1.0 } }
#         }
#         feature {
#           key: "custom_features_2"
#           value { float_list { value: 1.5 } }
#         }
#         feature {
#           key: "utility"
#           value { int64_list { value: 1 } }
#         }
#       }
#     }
#     examples {
#       features {
#         feature {
#           key: "custom_features_1"
#           value { float_list { value: 1.0 } }
#         }
#         feature {
#           key: "custom_features_3"
#           value { float_list { value: 2.1 } }
#         }
#         feature {
#           key: "utility"
#           value { int64_list { value: 0 } }
#         }
#       }
#     }""", input_pb2.ExampleListWithContext())


# def _write_tfrecord_files(path):
#   elwc_list = [ELWC.SerializeToString()] * 10
#   if tf.io.gfile.exists(path):
#     tf.io.gfile.remove(path)

#   with tf.io.TFRecordWriter(path) as writer:
#     for elwc in elwc_list:
#       writer.write(elwc)


# class TFRankingCannedDNNTest(tf.test.TestCase, parameterized.TestCase):

#   def setUp(self):
#     super(TFRankingCannedDNNTest, self).setUp()
#     tf.compat.v1.reset_default_graph()

#     # Prepares model directory, and train and eval data.
#     self._base_model_dir = tf.compat.v1.test.get_temp_dir() + "/model/"
#     tf.io.gfile.makedirs(self._base_model_dir)
#     self._data_file = os.path.join(self._base_model_dir, "elwc.tfrecord")
#     _write_tfrecord_files(self._data_file)

#   def tearDown(self):
#     super(TFRankingCannedDNNTest, self).tearDown()
#     if self._base_model_dir:
#       tf.io.gfile.rmtree(self._base_model_dir)
#     self._base_model_dir = None

#   @parameterized.named_parameters(("enable_listwise_inference", True),
#                                   ("disable_listwise_inference", False))
#   def test_train_and_eval(self, listwise_inference):
#     self._model_dir = self._base_model_dir + "/" + str(listwise_inference)
#     with flagsaver.flagsaver(
#         train_input_pattern=self._data_file,
#         eval_input_pattern=self._data_file,
#         model_dir=self._model_dir,
#         num_features=3,
#         num_train_steps=10,
#         listwise_inference=listwise_inference):
#       tf_ranking_canned_dnn.train_and_eval()


# if __name__ == "__main__":
#   tf.test.main()